# Веб-скрейпинг и таблицы

*Алла Тамбовцева, НИУ ВШЭ*

**Наша задача:** выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Импортируем её:

In [1]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`).

In [18]:
url = 'https://nplus1.ru/' # сохраняем
page = requests.get(url, ) # загружаем страницу по ссылке

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [19]:
page  # response 200 - страница загружена

<Response [200]>

Импортируем функцию `BeautifulSoup` из библиотеки `bs4` (от *beautifulsoup4*) и заберём со страницы `page` код html в виде текста. 

In [20]:
from bs4 import BeautifulSoup  # не спрашивайте, почему BeautifulSoup

In [21]:
soup = BeautifulSoup(page.text, 'lxml')

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U* в Google Chrome).

In [22]:
soup

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>N + 1: научные статьи, новости, открытия</title>
<link href="/front-build/css/main.css" rel="stylesheet"/>
<link href="/front-build/css/app.css" rel="stylesheet"/>
<link href="/images-new/favicon-bw.png" rel="icon"/>
<link href="https://nplus1.ru" rel="canonical"/>
<meta content="8c90b02c84ac3b72" name="yandex-verification"/>
<meta content="b419949322895fc9106e24ed01be58ac" name="pmail-verification"/>
<script>window.yaContextCb = window.yaContextCb || []</script>
<script async="" src="https://yandex.ru/ads/system/context.js"></script>
<meta content="N + 1: научные статьи, новости, открытия" property="og:site_name"/>
<meta content="N + 1: научные статьи, новости, открытия" property="og:title"/>
<meta content="https://nplus1.ru/images-new/share_snippet.png" property="og:image"/>
<meta content="https://nplus1.ru" proper

Для просмотра выглядит не очень удобно.  «Причешем» наш `soup` – воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [23]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   N + 1: научные статьи, новости, открытия
  </title>
  <link href="/front-build/css/main.css" rel="stylesheet"/>
  <link href="/front-build/css/app.css" rel="stylesheet"/>
  <link href="/images-new/favicon-bw.png" rel="icon"/>
  <link href="https://nplus1.ru" rel="canonical"/>
  <meta content="8c90b02c84ac3b72" name="yandex-verification"/>
  <meta content="b419949322895fc9106e24ed01be58ac" name="pmail-verification"/>
  <script>
   window.yaContextCb = window.yaContextCb || []
  </script>
  <script async="" src="https://yandex.ru/ads/system/context.js">
  </script>
  <meta content="N + 1: научные статьи, новости, открытия" property="og:site_name"/>
  <meta content="N + 1: научные статьи, новости, открытия" property="og:title"/>
  <meta content="https://nplus1.ru/images-new/share_snippet.png" property="og:ima

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране).

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Посмотрим на кусочки кода, соответствующие всем ссылкам на главной странице сайта:

In [24]:
for link in soup.find_all('a'):
    print(link.get('href'))

/search
https://offline.nplus1.ru/
https://nplus.pro/
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/blog/2022/04/01/samotek
/search/empty/869
/search/empty/874
/search/empty/880
/search/empty/768
/search/empty/890
/search/empty/871
/search/empty/876
/search/empty/775
/search/empty/767
/search/empty/771
/search/empty/772
/search/empty/778
/search/empty/917
/search/empty/918
/search/empty/824
https://t.me/nplusone
https://vk.com/nplusone
https://ok.ru/nplus1
https://twitter.com/nplusodin
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/news/2015/09/21/editor-thy-name
/search/empty/869
/search/empty/874
/search/empty/880
/search/empty/768
/search/empty/890
/search/empty/871
/search/empty/876
/search/empty/775
/search/empty/767
/search/empty/771
/search/empty/772
/search/empty/778
/search/empty/917
/search/empty/918
/search/empty/824
https://offline.nplus1.ru/
https://nplus.pro/
https://t.me/nplusone
h

Ссылок много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [35]:
urls = []

for link in soup.find_all('a'):
    if '/news' in link.get('href'):
        if 'https://' in link.get('href'):
            urls.append(link.get('href'))
urls

['https://nplus1.ru/news/2015/09/21/editor-thy-name',
 'https://nplus1.ru/news/2022/10/31/bombus-terrestris',
 'https://nplus1.ru/news/2022/11/01/nudes',
 'https://nplus1.ru/news/2022/11/01/why-cats-have-nine-lives',
 'https://nplus1.ru/news/2022/11/01/otus-bikegila',
 'https://nplus1.ru/news/2022/11/01/family-in-jar',
 'https://nplus1.ru/news/2022/11/01/mali-malaria-mab',
 'https://nplus1.ru/news/2022/10/31/echo-hubble',
 'https://nplus1.ru/news/2022/10/31/bombus-terrestris',
 'https://nplus1.ru/news/2022/10/31/marking-model',
 'https://nplus1.ru/news/2022/10/31/mars-samples',
 'https://nplus1.ru/news/2022/10/31/hubble-rocky-atmosphere',
 'https://nplus1.ru/news/2022/10/31/retina-moving',
 'https://nplus1.ru/news/2022/10/31/ephesos',
 'https://nplus1.ru/news/2022/10/31/sheep-collective-motion',
 'https://nplus1.ru/news/2022/10/31/mengtian-launch',
 'https://nplus1.ru/news/2022/10/31/iron-age-woman',
 'https://nplus1.ru/news/2022/10/29/sun-video',
 'https://nplus1.ru/news/2022/10/29/ps

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 1, у вас может быть другая, новости обновляются.

In [60]:
url0 = urls[15]

page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text, 'lxml')

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [61]:
soup0.find_all('meta')

[<meta charset="utf-8"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="8c90b02c84ac3b72" name="yandex-verification"/>,
 <meta content="b419949322895fc9106e24ed01be58ac" name="pmail-verification"/>,
 <meta content="N + 1: научные статьи, новости, открытия" property="og:site_name"/>,
 <meta content="Китай запустил в космос последний модуль орбитальной станции «Тяньгун»" property="og:title"/>,
 <meta content="https://app-images.website.yandexcloud.net/711812/635f7d4b5fafc_cover_share.jpg" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2022/10/31/mengtian-launch" property="og:url"/>,
 <meta content="Это лабораторный модуль «Мэньтянь»" property="og:description"/>,
 <meta content="article" property="og:type"/>,
 <meta content="2022-10-31" itemprop="datePublished"/>,
 <meta content="2022-10-31" itemprop="datePublished"/>,
 <meta content="Александр Войтюк" name="mediator_author"/>,
 <meta content="Александр Войтюк" name="author"/>,
 <me

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута: 

In [45]:
soup0.find_all('meta', {'name' : 'author'}) # например, автор

[<meta content="Михаил Подрезов" name="author"/>]

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [46]:
soup0.find_all('meta', {'name' : 'author'})[0]

<meta content="Михаил Подрезов" name="author"/>

Нам нужно вытащить из этого объекта `content` – имя автора. Посмотрим на атрибуты:

In [47]:
soup0.find_all('meta', {'name' : 'author'})[0].attrs

{'name': 'author', 'content': 'Михаил Подрезов'}

Как получить отсюда `content`? Очень просто, ведь это словарь! А доставать из словаря значение по ключу мы умеем.

In [48]:
author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
author

'Михаил Подрезов'

Аналогичным образом извлечем дату, заголовок и описание.

In [56]:
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
description = soup0.find_all('meta', {'property' : 'og:description'})[0].attrs['content']

Не прошло и двух пар, как мы разобрались со всем :) Теперь осталось совсем чуть-чуть. Написать готовую функцию для всех проделанных нами действий и применить ее в цикле для всех ссылок в списке `full_urls`. Напишем! Аргументом функции будет ссылка на новость, а возвращать она будет текст новости и всю необходимую информацию (дата, автор, сложность и проч.). Скопируем все строки кода выше.

In [76]:
def GetNews(url0):
    """
    Returns a tuple with url0, date, author, description, title, final_text, rubrics, diff.
    Parameters:
    
    url0 is a link to the news (string)
    """
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'lxml')
    
    author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
    date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
    description = soup0.find_all('meta', {'property'  : 'og:description'})[0].attrs['content']
 
    
    return url0, date, author, description, title

Уфф. Осталось применить ее в цикле. Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [77]:
from time import sleep

In [89]:
len(urls)

81

In [92]:
news = [] # это будет список из кортежей, в которых будут храниться данные по каждой новости

for link in urls[:50]:
    res = GetNews(link)
    news.append(res)
    sleep(1) # задержка в 3 секунды

In [93]:
news[0]

('https://nplus1.ru/news/2015/09/21/editor-thy-name',
 '2015-09-21',
 'Андрей Коняев',
 '',
 'Мы ищем редакторов: физиков, археологов, медиков')

Так теперь выглядит первый элемент списка:

Импортируем `pandas` и создадим датафрейм из списка кортежей: 

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(news)

In [ ]:
df.head(10)

Переименуем столбцы в базе.

In [ ]:
df.columns = ['link', 'date', 'author', 'desc', 'title', 'text', 'rubric', 'diffc']

In [ ]:
df.head(2)

Теперь внесем изменения: сделаем столбец `diffc` числовым – типа *float*.

In [ ]:
df['diffc'] = [float(i) for i in df.diffc]

Теперь сложность представлена в базе как количественный показатель, и описывать ее можно соответствующим образом:

In [ ]:
df.diffc.describe()

Теперь столбец со сложностью точно числовой. Можем даже построить для него гистограмму.

In [ ]:
%matplotlib inline
df.diffc.plot.hist()

Объединим рубрики в *text* в одну строку через запятую:

In [ ]:
df['rubric'] = [','.join(r) for r in df.rubric]

Давайте почистим текст новостей – уберем оттуда текст, не относящийся к новостям. Найдем лишнее:

In [ ]:
df.text[0]

Лишний текст находится после фразы 'Нашли опечатку?'. Так давайте будем разбивать строки по этой фразе с помощью метода `.split()` и брать все, что до нее (элемент с индексом 0).

In [ ]:
df['clean_text'] = [t.split('Нашли опечатку?')[0] for t in df.text]

Осталось только заменить непонятные символы `\xa0` на пробелы:

In [ ]:
df['clean_text'] = [t.replace("\xa0", " ") for t in df.clean_text]

In [ ]:
df.clean_text[0]

Всё! Сохраняем датафрейм в файл. Для разнообразия сохраним в Excel:

In [ ]:
df.to_excel('nplus-news.xlsx')